In [3]:
import pickle
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [27]:
with open('neutral_politics_top_100_comment_bodies_clean.pkl', 'rb') as f:
    comments = pickle.load(f)

In [28]:
len(comments)

30530

In [29]:
unq_comments = list(set(comments))

In [30]:
len(unq_comments)

30391

In [31]:
comments[100]

' technicality, but ruled unconstitutional as practiced by nypd  not a technicality. stop and frisk is as unconstitutional as the death penalty  the death penalty was temporarily suspended as unconstitutional as practiced, but was reinstated when it was thought that it could be adequately implemented in a way that respected the 14th amendment. same with stop and frisk  suspended in a particular case, but not essentially unconstitutional.'

In [32]:
from collections import defaultdict

word_tokenizer = nltk.tokenize.TreebankWordTokenizer()

following = defaultdict(list)
count = defaultdict(int)
start_list = []
for comment in unq_comments:
    words = word_tokenizer.tokenize(comment)
    if words:
        start_list.append(words[0])
    for index, word in enumerate(words[:-1]):
        following[word].append(words[index + 1])
        count[word] += 1
    
    if words:
        last_word = words[-1]
        if last_word not in following:
            following[last_word] = []
            count[last_word] = 0

In [33]:
bigrams = defaultdict(list)
start_bigrams = []
for comment in unq_comments:
    words = word_tokenizer.tokenize(comment)
    if len(words) > 1:
        start_bigrams.append((words[0], words[1]))
    for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
        bigram = (w1,w2)
        bigrams[bigram].append(w3)

In [62]:
import random
import numpy as np

# Based on code from pairprogramming solution
def generate_text_unigram(start_list, following_dict, num_words):
    
    w = [np.random.choice(start_list)]
    #w[0] = w[0][0].upper() + w[0][1:]
    
    for _ in range(num_words - 1):
        next_word = np.random.choice(following_dict[w[-1]])
        w.append(next_word)
    return ' '.join(w)


In [68]:
def generate_text_bigram(start_bigrams, following_dict, bigram_dict, num_words):
    # Based on code from pair programming solution
    
    all_bigrams = list(bigram_dict.keys())
    current_bigram = random.choice(start_bigrams)
    text = list(current_bigram)
    text[0] = text[0][0].upper() + text[0][1:]
    
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            next_word = random.choice(bigram_dict[current_bigram])
        else:
            next_word = random.choice(following_dict[text[-1]])
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)
    

In [36]:
most_popular_word = sorted(count.items(), key = lambda x: x[1], reverse=True)[0][0]

print("The most popular word is: " + most_popular_word)

for word in following:
    if len(following[word]) == 0:
        following[word] = [most_popular_word]

The most popular word is: the


In [66]:
generate_text_unigram(start_list, following, 20)

"replied to do not we have nearly half didn't. they are times in on the 2016 . the media and"

In [83]:
generate_text_bigram(start_bigrams, following, bigrams, 100)

"Case does n't improve no matter the target. the plea agreement . now please stand quietly for a big payout and i still take issue with it. before we started privatizing and cutting out the fact veselnitskaya was a natural monopoly , or information at the same people if you own the name. neutrality. do n't imagine that the two who did n't do that either. it feels good but this explanation is misleading to correlate the gun by bragging about how there could be a great place , so exactly the sort of speaking straight out of pocket. in canada"

In [1]:
import gensim

google_vec_file = '~/Downloads/GoogleNews-vectors-negative300.bin'


In [2]:
google_model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [39]:
## Generation method for returning most relevant comment to given input. Gotta do Doc2Vec!

def generate_most_relevant_comment(input_string):
    most_similar_score = 0
    most_similar = ''
    for comment in comments:
        similarity_score = google_model.wv.similarity(input_string, comment)
        if similarity_score > most_similar_score:
            most_similar_score = similarity_score
            most_similar = comment
    return comment

In [40]:
string = "Let's talk about gun control. The US has the least restrictive gun control laws in the developed world. Is this a good thing or a bad thing"
generate_most_relevant_comment(string)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


KeyError: "word 'Let's talk about gun control. The US has the least restrictive gun control laws in the developed world. Is this a good thing or a bad thing' not in vocabulary"

In [50]:
test_list = ['the']
test_list[0]

'the'

In [51]:
test_list[0] = test_list[0][0].upper() + test_list[0][1:]

In [52]:
test_list[0]

'The'